In [2]:
import os 
from skimage import io 
from deepcell.applications import CellTracking
from tifffile import imread
import csv
import tifffile 

import copy
import os

import imageio
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np

from deepcell.applications import NuclearSegmentation, CellTracking
from deepcell_tracking.trk_io import load_trks
  
def shuffle_colors(ymax, cmap):
    """Utility function to generate a colormap for a labeled image"""
    cmap = mpl.colormaps[cmap].resampled(ymax)
    nmap = cmap(range(ymax))
    np.random.shuffle(nmap)
    cmap = ListedColormap(nmap)
    cmap.set_bad('black')
    return cmap

def tif_to_nparray(filepath):
    # Read the TIF file
    tif_data = imread(filepath)

    # Convert the TIF data to a NumPy array
    np_array = np.array(tif_data)

    return np_array

In [51]:
OUTPUT_DIR = '/Users/lian/Desktop'

# (raw image)
folder_path = '/Users/lian/Desktop/XY002.tif'

# (segmented image)
folder_path1 = '/Users/lian/Desktop/masks2.tif'

In [1]:
# raw image array
y = tif_to_nparray(folder_path) 
y1 = y[..., np.newaxis]  

print("raw image converted", y1.shape)  

# segmented image array 
X = tif_to_nparray(folder_path1)
X1 = X[..., np.newaxis]  

print("segmented image converted", X1.shape)

NameError: name 'tif_to_nparray' is not defined

In [ ]:
# Tracking = CellTracking(model=None, neighborhood_encoder=None, distance_threshold=64, appearance_dim=32, birth=0.99, death=0.99, division=0.01, track_length=8, embedding_axis=0, crop_mode='resize', norm=True)
# ^above is default parameters of the Tracker
tracker = CellTracking()

tracked_data = tracker.track(image = y1, labels = X1)
y_tracked = tracked_data['y_tracked']

In [111]:
y_tracked = tracked_data['y_tracked']
# Save as a TIFF image
tifffile.imsave(OUTPUT_DIR+"/test-2223.tif", Dict['y_tracked'])

In [ ]:
# Save as csv

# Extract keys and values from the dictionary 
keys = Dict["tracks"].keys()
values = Dict["tracks"].values() 

output_file = 'data2.csv'

with open(OUTPUT_DIR + "/" + output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerow(values)

In [44]:
#save as gif

ymax = np.max(y_tracked)
cmap = shuffle_colors(ymax, 'tab20')

def plot(x, y):
    yy = copy.deepcopy(y)
    yy = np.ma.masked_equal(yy, 0)
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(x, cmap='Greys_r')
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(yy, cmap=cmap, vmax=ymax)
    ax[1].set_title('Tracked')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image

imageio.mimsave(
    './track.gif',
    [plot(y1[i,...,0], y_tracked[i,...,0])
     for i in range(y_tracked.shape[0])]
)